* By passenger, we mean someone in the building who is waiting to use or is currently using the elevator.
* The user of your classes should be able to customize the number of floors and passengers in the building.
* The passenger’s starting floor should be random, and they should have a random destination. (It’s up to you to determine what random should mean here.)
* Each passenger uses the elevator only once.
* Each passenger uses the elevator only once.
* (Would be nice) The program should have error checking to ensure valid user input.

In [6]:
from random import randint
from copy import deepcopy

class Elevator(object):
    """An elevator class.
    It has position, passengers inside and direction.
    """

    def __init__(self, n_floors, n_passengers, capacity = 15, verbose = True):
        self.num_of_floors = n_floors
        self.current_floor = 0
        self.direction = True
        self.register_list = []
        self.capacity = capacity
        self.strategy = 0
        self.passengers_num = n_passengers
        self.verbose = verbose
 
        self.waiting_list = []
        for i in range(self.passengers_num):
            self.waiting_list.append(Passenger(i, self.num_of_floors))

        # self.waiting_list = sorted(self.waiting_list, key=lambda x: x.start_floor)
        # sorted for performance reason

    def move(self):
        """Method for elevator movement.
        In every step elevator either goes a floor higher or a floor lower.
        """
        self.current_floor += 1 if self.direction else -1

    def enter_passengers(self):
        """Method to get in all passengers and erase them from the list
        """
        for passenger in self.waiting_list:
            if passenger.start_floor == self.current_floor and len(self.register_list) < self.capacity:
                self.register_list.append(passenger)
                self.waiting_list.remove(passenger)
                if self.verbose: print ("Elevator picks up passenger %d on floor %d"% (passenger.ID, passenger.start_floor))

    def exit_passengers(self):
        """Function to remove all passengers on their destination floor.
        It is called in run method of the building class.
        """
        for passenger in self.register_list:
            if passenger.dst_floor == self.current_floor:
                self.register_list.remove(passenger)
                if self.verbose: print("Elevator drops off passenger %d on floor %d"% (passenger.ID, passenger.dst_floor))
    """
    def direction_default_strategy(self):
        """Default function of elewator work - 
        it starts with going to the roof of building, then
        comes back to the first floor.
        """
        if self.current_floor >= self.num_of_floors - 1:
            self.direction = False
        elif self.current_floor <= 0:
            self.direction = True

    def direction_bad_strategy(self):
        """Example function of very bad strategy.
        Elevator takes desired floor of the first
        passenger and elevates him to desired location,
        then repeats for the next passenger.
        If no passenger is in elevator it just goes to the up and down,
        as in default strategy.
        """
        if len(self.register_list) is 0:
            self.direction_default_strategy()
            return
        firstval = self.register_list[0].dst_floor
        if self.current_floor > firstval:
            self.direction = False
        else:
            self.direction = True


In [7]:
class Passenger(object):
    """A passenger class. 
    A passenger has his or her ID number, starting floor and destination floor number
    """

    def __init__(self, ID, floorsNum):
        """Initializes values for passenger and select random floors to start
        Destination floor and starting floor differ
        """
        self.ID = ID
        self.start_time = 0 # For wait time calculation
        self.start_floor = randint(0, floorsNum-1)  #works for now, would be interesting to use different distributions
        self.dst_floor = randint(0, floorsNum-2)    #same as above

        if self.dst_floor == self.start_floor:
            self.dst_floor += 1 # this will bias the destination floors upward

        # Lets add an attribute for wait_time here and increase that until they reach destination


In [10]:
class Building(object):
    """A building class.
    Building had number of floors, list of passengers outside the elevator and its own elevator object.
    Strategy of an elevator is set by integer, where 0 is default and 1 is another very naive strategy.
    """

    def __init__(self, verbose = True):
        """Makes building class and addes lust of passengers
        The list is sorted by starting floor value for efficacy.
        """
        self.num_of_floors = self.get_value("Please write number of floors in the building: ",\
                                   "Incorrect value. Number of floors should be integer higher than 1.", 2)

        passengers_num = self.get_value("Please write number of passengers: ",\
                                    "Incorrect value. Number of passengers should be non-negative integer.", 0)
        self.elevator = Elevator(self.num_of_floors,passengers_num, verbose)

    def get_value(self,message,incorret_message,minimal):
        """Method for making sure to obtain integers
        """
        val = None
        try:
            val = int(raw_input(message))
        except ValueError:
            print incorret_message
            return self.get_value(message, incorret_message, minimal)
        if val < minimal:
            print incorret_message
            return self.get_value(message, incorret_message, minimal)
        else:
            return val

    def run(self):
        """Step function. When called:
        1. waiting passengers enter the elevator (register_passenger)
        2. the elevator is assigned a direction value
        3. elevator moves one floor up or down
        4. passengers on destination floors leaves the elevator (cancel_passenger)
        """
        self.elevator.enter_passengers()
        # self.elevator.strategy (attribute of the building class)
        if self.elevator.strategy == 0:
            self.elevator.direction_default_strategy()
        else:
            self.elevator.direction_bad_strategy()
        self.elevator.move()
        self.elevator.exit_passengers()

    def output(self):
        """Return total number of steps taken
        """
        total_steps = 0 
        while self.elevator.waiting_list or self.elevator.register_list:
            self.run()
            total_steps += 1
        print (total_steps)


In [19]:
def direction_default_strategy(self):
        """Default function of elewator work - 
        it starts with going to the roof of building, then
        comes back to the first floor.
        """
        if self.current_floor >= self.num_of_floors - 1:
            self.direction = False
        elif self.current_floor <= 0:
            self.direction = True
            
def direction_bad_strategy(self):
        """Example function of very bad strategy.
        Elevator takes desired floor of the first
        passenger and elevates him to desired location,
        then repeats for the next passenger.
        If no passenger is in elevator it just goes to the up and down,
        as in default strategy.
        """
        if len(self.register_list) is 0:
            self.direction_default_strategy()
            return
        firstval = self.register_list[0].dst_floor
        if self.current_floor > firstval:
            self.direction = False
        else:
            self.direction = True

In [18]:
 def main():
    """Main function"""
    building_def = Building(verbose = True)
    building_bad = deepcopy(building_def)
    print "Number of steps for default strategy:", building_def.output()

    building_bad.elevator.strategy = 1
    print "Number of steps for bad strategy:", building_bad.output()

if __name__ == "__main__":
    main()

Please write number of floors in the building: 2
Please write number of passengers: 2
 Number of steps for default strategy: Elevator picks up passenger 0 on floor 0
Elevator drops off passenger 0 on floor 1
Elevator picks up passenger 1 on floor 1
Elevator drops off passenger 1 on floor 0
2
None
1
Number of steps for bad strategy: Elevator picks up passenger 0 on floor 0
Elevator drops off passenger 0 on floor 1
Elevator picks up passenger 1 on floor 1
Elevator drops off passenger 1 on floor 0
2
None
